*Copyright statement:
This material, no matter whether in printed or electronic form, may be used for personal and non-commercial educational use only. Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.*

# Polynomial Regression

The goal of polynomial regression (or also linear regression) is to find the function which best models the relationship between an independent variable $x$ and a dependent variable $y$ as an $n$-th degree polynomial. With polynomial regression, this means finding the coefficients that provide the best fit to a non-linear relationship in the data, which a straight line (linear regression) cannot always do precisely.

We model this relationship through the polynomial:

$$y \approx \theta_0 + \theta_1 x + \theta_2 x^2 + \theta_3 x^3 + ... + \theta_n x^n$$

where the thetas $\theta_0, \theta_1, ..., \theta_n$ are the parameters we are trying to find. Throughout this notebook, we will:

*    Generate synthetic, noisy data we are trying to fit a function to
*    Create a polynomial feature matrix to make a linear model fit non-linear data
*    Finding the optimal weights through least squares
*    Evaluating the generalization error and empirical risk through the Mean Squared Error
* Intercatively visualize the impact of different settings

We first import all necessary packages:


In [ ]:
!pip install ipympl

In [139]:
# necessary imports
import torch

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from ipywidgets import interact
%matplotlib widget

Next, we set a seed for deterministic data generation:

In [ ]:
# seed for reproducibility
seed = 42
torch.manual_seed(seed)

### Data generation

Now let us generate our data. We define the sine function as our true underlying function and generate some datapoints around it. To make it look more like real data you could have gained from an experiment, we add some Gaussian noise as well. In general, training on realistic/noisy data also makes sure that our model becomes more robust and can generalize even if outliers are present, as it helps simulate the unpredictability of the real world.

We first create a function `true_polynomial` which models our sine function. Here $x \in \mathbb{R}$ is expected to be between 0 and 1, so we obtain a sine wave that completes a full period ($2 \pi$) between 0 and 1.

In [141]:
def true_polynomial(x):
    return torch.sin(2 * torch.pi * x)

We also provide the functionality for generating datapoints `generate_datapoints`, which takes in `n_samples` the number of datapoints we want to create around our true polynomial, and `noise` the degree of noise in our data, aka the spread of our data around the true function.

In [142]:
def generate_datapoints(n_samples, noise):
    X = torch.rand(n_samples, 1) # n_samples in [0,1)
    # obtain noisy observations around the true polynomial
    noisy_y = noise * torch.randn(n_samples) # generate gaussian noise 
    y = true_polynomial(X.squeeze()) + noisy_y # y = y + noise
    return X, y

As previously mentioned, our relationship is of the form:

$$y \approx \theta_0 + \theta_1 x + \theta_2 x^2 + \theta_3 x^3 + ... + \theta_n x^n$$

However, linear regression can only fit a straight line. If we thus have data of the form above, like curved data or a parabola, we need a polynomial regression model that solves this problem by adding powers of $x$ as new inputs. That way, the model can bend to fit more complex shapes of relationships in the data. For this to work, we first need to transform our inputs $x$ to polynomial features $[1, x, x^2, x^3,...,x^d]$, depending on the degree of the polynomial $d$. With this, we transform our problem such that a linear model can actually fit non-linear data. In `scikit-learn`, this is done through [`Polynomial Features`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html), however you will implement this manually.

### Task 1: Implement polynomial feature expansion

Given your inputs $x$, generate a new feature matrix consisting of all polynomial combintions of $x$. For example, if you have an input $[a,b]$, the degree-2 polyomial features are $[1, a, b, a^2, ab, b^2]$.

In [143]:
def build_polynomial_features(X, degree):
    #TODO: implement polynomial feature expansion
    # YOUR CODE HERE
    raise NotImplementedError
    return torch.cat(features, dim=1)

### Task 2: Fitting the model

Now that you have expanded the input features to their polynomial features, you now want to find the best weight vector $\theta$, such that multiplying it with our polynomial features comes as close as possible to our true outputs $y$. One of the ways we can solve this problem is the Least Squares method, which minimizes the squared errors between the predictions and real outputs. This is given by:

$$\theta = (X^T X)^{-1} X^T y$$

where $X$ is your polynomial feature matrix and $y$ are the true target values. This is also a closed-form solution to linear regression. In this task, you should implement the `fit_leat_squares` function, which takes as arguments `X_poly`, your matrix of polynomial features, and the true values `y`. It should implement the above formula and return the optimal $\theta$ vector.

In [145]:
def fit_least_squares(X_poly, y):
    #TODO implement least squares
    # YOUR CODE HERE
    raise NotImplementedError

### Task 3: Predict outputs

Finally, using a linear approach, we predict our $\hat y$ as:

$$\hat y = X_{poly} \theta$$

Implement the `predict` function below that does exactly this operation, given `X_poly` as the polynomial feature matrix, and `weights` as the thetas, returning $\hat y$.

In [147]:
def predict(X_poly, weights):
    # TODO: implement prediction here
    # YOUR CODE HERE
    raise NotImplementedError

### Task 4: Evaluating the model

Finally, after you found the optimal weights and made predictions, it is time to measure how good these predictions are. This is what the Mean Squared Error (MSE) is for, which measures how far off your predictions are from the true output. Hence, a smaller MSE is better. We will use the MSE to calculate our empirical risk (error over our training set) and our generalization error (error over our test set).

The MSE is measured as:

$$MSE = \frac{1}{n} \sum^n_{i=1} (y_i - \hat y_i)^2$$

where $n$ is the number of datapoints, $y_i$ is the $i$'th true ouput and $\hat y_i$ is the prediction the model makes for the $i$'th data point. Implement the MSE calculation in the `mean_squared_error` function:

In [149]:
def mean_squared_error(y_true, y_hat):
    # TODO: Implement MSE formula
    # YOUR CODE HERE
    raise NotImplementedError

### Putting it all together

Below we provide a function `plot_polyfit`, which interactively puts your code snippets into a polynomial regression pipeline. It first generates some data with the specified amount of datapoints and noise, then uses `scikit-learn`'s `train_test_split` to split the generated data points into training and test set, builds the polynomial feature matrix out of $x$ and calculates the optimal weights $\theta$. Finally, it predicts the $\hat y$ and computes the training error and test errors. 

In [151]:
def plot_polyfit(n_samples=30, noise=0.1, degree=3):
    # generating data point
    x, y = generate_datapoints(n_samples=n_samples, noise=noise)

    # split data points into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(x.numpy(), y.numpy(), test_size=0.2, random_state=seed)
    X_train = torch.tensor(X_train)
    X_test = torch.tensor(X_test)
    y_train = torch.tensor(y_train)
    y_test = torch.tensor(y_test)

    # building polynomial feature matrix from the X's
    X_train_poly = build_polynomial_features(X_train, degree)
    X_test_poly = build_polynomial_features(X_test, degree)
    
    # compute optimal weights via least squares method
    weights = fit_least_squares(X_train_poly, y_train)
    
    # prepare predictions
    x_plot = torch.linspace(0, 1, steps=100).unsqueeze(1)
    x_plot_poly = build_polynomial_features(x_plot, degree)
    y_plot = predict(x_plot_poly, weights)
    y_train_pred = predict(X_train_poly, weights)
    y_test_pred = predict(X_test_poly, weights)
    
    # compute emp risk and generalization error
    train_error = mean_squared_error(y_train, y_train_pred)
    test_error = mean_squared_error(y_test, y_test_pred)
    
    # plotting
    plt.figure(figsize=(8, 5))
    plt.scatter(X_train, y_train, label="Training data", alpha=0.7)
    plt.scatter(X_test, y_test, label="Test data", color="orange")
    plt.plot(x_plot, y_plot.detach(), 'r-', label=f"Polynomial degree {degree}")
    plt.plot(x_plot, true_polynomial(x_plot.squeeze()).detach().numpy(), label="True function", color="green", linestyle="dashed")
    plt.ylim(-1.5, 1.5)
    plt.legend()
    plt.title(f"Train error: {train_error:.3f} | Test (generalization) error: {test_error:.3f}")
    plt.grid(True)
    plt.show()

Feel free to execute the code below and play around with the sliders in oder to increase/decrease e.g. the number of datapoints as well as the degree of the polynomial that should be predicted to fit the data. You should notice that very low datasamples paired with a higher-degree polynomial will lead to so-called overfitting (try e.g. n_samples=15 and degree=11): While a very high-degree polynomial can fit most/all training set datapoints perfectly, its ability to generalize to the unseen test set points is very bad, leading to a small training error, but a high generalization error.

In [ ]:
interact(plot_polyfit, 
         n_samples=(5, 100, 5),
         noise=(0.0, 0.7, 0.1),
         degree=(0, 15, 1));